In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import librosa
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
#from utils import process_data,read_data
import random
import re
import sklearn
import sklearn.ensemble as en
import sklearn.linear_model as lm
import sklearn.neighbors as nei
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from bc_audio.utils import extract_raw_data, DIR_PATH, PROCESSED_AUD_FILE_PATH, FEATURE_SETS, generate_train_val_test_idx, calculate_random_baseline_from_splits
from bc_video.utils import extract_from_raw_data, process_data
from bc_video.binary_classifier_reworked import MODELS

In [16]:
pt = '/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/bc_video/inference/mlp/experiment-2/models/run-2/best_model_19.pth.tar'
pt2 = '/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/bc_audio/inference/mlp/experiment-4/models/run-0/best_model_0.pth.tar'
ckpt = torch.load(pt2)

In [17]:
for key, val in ckpt['net'].items():
    print(key, val.shape)

layer1.weight torch.Size([10, 12])
layer1.bias torch.Size([10])
layer2.weight torch.Size([10, 10])
layer2.bias torch.Size([10])
layer3.weight torch.Size([1, 10])
layer3.bias torch.Size([1])
bc10.weight torch.Size([10])
bc10.bias torch.Size([10])
bc10.running_mean torch.Size([10])
bc10.running_var torch.Size([10])
bc10.num_batches_tracked torch.Size([])


In [9]:
l=l = [module for module in ckpt['net'].modules() if not isinstance(module, nn.Sequential)]
              
print(l)

AttributeError: 'collections.OrderedDict' object has no attribute 'modules'

In [2]:
# Audio based model

Aud_model_path = '/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/bc_audio/inference/mlp/experiment-7/models/run-0/best_model_0.pth.tar'

# class Aud_Classifier(nn.Module):
#     def __init__(self, number_of_features):
#         super().__init__()
#         self.layer1 = nn.Linear(number_of_features, 10)                        
#         self.layer2 = nn.Linear(10, 10)
#         self.layer3 = nn.Linear(10, 1)

#         self.bc10 = nn.BatchNorm1d(10)
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout1d(p=0.1)

#         nn.init.xavier_normal_(self.layer1.weight) 
#         nn.init.xavier_normal_(self.layer2.weight)
#         nn.init.xavier_normal_(self.layer3.weight)

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.bc10(x)
#         x = self.relu(x)
#         x = self.layer2(x)
#         x = self.bc10(x)
#         x = self.relu(x)
#         x = self.dropout(x)        
#         x = self.layer3(x)
        
#         return x
class Aud_Classifier(nn.Module):
    def __init__(self, number_of_features):
        super().__init__()
        self.layer1 = nn.Linear(number_of_features, 20)                        
        self.layer2 = nn.Linear(20, 20)
        self.layer3 = nn.Linear(20, 1)

        self.bc10 = nn.BatchNorm1d(20)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout1d(p=0.2)

        nn.init.xavier_normal_(self.layer1.weight) 
        nn.init.xavier_normal_(self.layer2.weight)
        nn.init.xavier_normal_(self.layer3.weight)

    def forward(self, x):
        x = self.layer1(x)
        x = self.bc10(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.bc10(x)
        x = self.relu(x)
        #x = self.dropout(x)        
        x = self.layer3(x)
        
        return x
    
aud_model = Aud_Classifier(1)

checkpt = torch.load(Aud_model_path)

In [3]:
aud_model.load_state_dict(checkpt['net'])

<All keys matched successfully>

In [4]:
# Video based model

vid_model_path = '/media/chagan/2023-ELECTRA-field-MarcSchoolStudy/bc_video/inference/mlp/experiment-3/models/run-7/best_model_998.pth.tar'

# class Vid_Classifier(nn.Module):
#     def __init__(self, number_of_features):
#         super().__init__()
#         self.layer1 = nn.Linear(number_of_features, 10)                        
#         self.layer2 = nn.Linear(10, 10)
#         self.layer3 = nn.Linear(10, 1)

#         self.bc10 = nn.BatchNorm1d(10)
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout1d(p=0.2)

#         nn.init.xavier_normal_(self.layer1.weight) 
#         nn.init.xavier_normal_(self.layer2.weight)
#         nn.init.xavier_normal_(self.layer3.weight)

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.bc10(x)
#         x = self.relu(x)
#         x = self.layer2(x)
#         x = self.bc10(x)
#         x = self.relu(x)
#         x = self.dropout(x)        
#         x = self.layer3(x)
        
#         return x
class Vid_Classifier(nn.Module):
    def __init__(self, number_of_features):
        super().__init__()
        self.layer1 = nn.Linear(number_of_features, 10)                        
        self.layer2 = nn.Linear(10, 10)
        self.layer3 = nn.Linear(10, 1)

        self.bc10 = nn.BatchNorm1d(10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout1d(p=0.2)

        nn.init.xavier_normal_(self.layer1.weight) 
        nn.init.xavier_normal_(self.layer2.weight)
        nn.init.xavier_normal_(self.layer3.weight)

    def forward(self, x):
        x = self.layer1(x)
        x = self.bc10(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.bc10(x)
        x = self.relu(x)
        x = self.dropout(x)        
        x = self.layer3(x)
        
        return x

vid_model = Vid_Classifier(17)

checkpt = torch.load(vid_model_path)

In [5]:
vid_model.load_state_dict(checkpt['net'])

<All keys matched successfully>

In [6]:
vid_features = [
    #'gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z', 'gaze_angle_x', 'gaze_angle_y'
    'AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU10_r', 'AU12_r', 'AU14_r', 'AU15_r', 'AU17_r', 'AU20_r', 'AU23_r', 'AU25_r', 'AU26_r', 'AU45_r'
]
    
vid_extracted_data = extract_from_raw_data(vid_features, summary_stat='None', timestamp=True)

Data already exists in disk, reading..



In [7]:
aud_feat = [
    #     {'Name': 'emob', 'features' : [ 
    #                'mfcc_sma[1]',
    #             'mfcc_sma[2]',
    #             'mfcc_sma[3]',
    #             'mfcc_sma[4]',
    #             'mfcc_sma[5]',
    #             'mfcc_sma[6]',
    #             'mfcc_sma[7]',
    #             'mfcc_sma[8]',
    #             'mfcc_sma[9]',
    #             'mfcc_sma[10]',
    #             'mfcc_sma[11]',
    #             'mfcc_sma[12]'
    #            ]
    # }
        {'Name': 'emob', 'features' : [ 
                    'pcm_zcr_sma',
               ]
    }
]

aud_features = aud_feat[0]['features']
for featureset in aud_feat:

    print(f"Feature set: {featureset['features']}\n")

    extractor = FEATURE_SETS[featureset['Name']]['extractor']
    #features = feature_set_items["features"](extractor)
    
    features = featureset['features'] #Custom features

    aud_extracted_data = extract_raw_data(extractor, featureset['Name'], features, custom_window_length_factor=4, timestamp=True)

Feature set: ['pcm_zcr_sma']

Data already exists in disk, reading..



In [8]:
concat_data = pd.merge(vid_extracted_data, aud_extracted_data, on=[x for x in vid_extracted_data.columns if x not in vid_features], how='inner')

In [9]:
concat_features = aud_features + vid_features

In [10]:
concat_features

['pcm_zcr_sma',
 'AU01_r',
 'AU02_r',
 'AU04_r',
 'AU05_r',
 'AU06_r',
 'AU07_r',
 'AU09_r',
 'AU10_r',
 'AU12_r',
 'AU14_r',
 'AU15_r',
 'AU17_r',
 'AU20_r',
 'AU23_r',
 'AU25_r',
 'AU26_r',
 'AU45_r']

In [11]:
concat_data.drop(columns=['timestamp'], inplace=True)


In [12]:
def process_concat_data(data, vid_features, aud_features, split_ratio):

    pair_id_list = list(set(data["pair_id"]))  # get unique pair_ids

    flag=True

    tot_size=data.shape[0]

    #Rejection sampling
    split_stats={}

    while flag:

        random.shuffle(pair_id_list)
        split_index = round(split_ratio * len(pair_id_list))
        train_pair_ids = pair_id_list[: split_index]
        test_pair_ids = pair_id_list[split_index:]
        
        train_length = data.loc[
        data["pair_id"].isin(pair_id_list[: split_index]) #split ratio
        ].reset_index(drop=True).shape[0]

        df_sliced_with_train_pid = data.loc[data["pair_id"].isin(train_pair_ids)].reset_index(drop=True)
        df_sliced_with_test_pid = data.loc[data["pair_id"].isin(test_pair_ids)].reset_index(drop=True)
        train_size = df_sliced_with_train_pid.shape[0]

        test_p_size = len([p for p in df_sliced_with_test_pid['pair_id'] if p[0] == 'P'])
        test_n_size = len([n for n in df_sliced_with_test_pid['pair_id'] if n[0] == 'N'])        
        
        condition_1 = train_size/tot_size > (split_ratio-0.02)
        condition_2 = train_size/tot_size < (split_ratio+0.02)
        condition_3 = len([d for d in pair_id_list[ : split_index] if d[0]=='N']) - len([d for d in pair_id_list[: split_index] if d[0]=='P']) < 2
        condition_4 = 0.5 <= test_p_size / df_sliced_with_test_pid.shape[0] and test_p_size / df_sliced_with_test_pid.shape[0] <= 0.52
        
        if condition_1 and  condition_2 and condition_3 and condition_4:
            flag=False
    
    split_stats['Split ratio']=train_length/tot_size
    split_stats['IDs in Train']=pair_id_list[: split_index]
    split_stats['IDs in Test']=pair_id_list[split_index : ]
    split_stats['Number of Ns in train']=len([d for d in pair_id_list[: split_index] if d[0]=='N'])
    split_stats['Number of Ps in train']=len([d for d in pair_id_list[: split_index] if d[0]=='P'])
    split_stats['Number of Ns in test']=len([d for d in pair_id_list[split_index :] if d[0]=='N'])
    split_stats['Number of Ps in test']=len([d for d in pair_id_list[split_index :] if d[0]=='P'])

    x_train = data.loc[
        data["pair_id"].isin(pair_id_list[: split_index])
    ].reset_index(drop=True)

    x_test = data.loc[
        data["pair_id"].isin(pair_id_list[split_index :])
    ].reset_index(drop=True)
    
    scaler_a, scaler_v=StandardScaler(), StandardScaler()
    scaler_a.fit(data[aud_features])
    scaler_v.fit(data[vid_features])

    x_train_a = scaler_a.transform(x_train[aud_features])
    x_test_a = scaler_a.transform(x_test[aud_features])

    x_train_v = scaler_v.transform(x_train[vid_features])
    x_test_v = scaler_v.transform(x_test[vid_features])
    
    y_train = data.loc[
        data["pair_id"].isin(pair_id_list[: split_index])
    ].reset_index(drop=True)
    y_train = y_train["condition"].apply(lambda x: 1.0 if (x == "positive") else 0.0)

    y_test = data.loc[
        data["pair_id"].isin(pair_id_list[split_index :])
    ].reset_index(drop=True)
    y_test = y_test["condition"].apply(lambda x: 1.0 if (x == "positive") else 0.0)
    
    x_train_a = torch.tensor(x_train_a, dtype=torch.float32)
    x_test_a = torch.tensor(x_test_a, dtype=torch.float32)

    x_train_v = torch.tensor(x_train_v, dtype=torch.float32)
    x_test_v = torch.tensor(x_test_v, dtype=torch.float32)

    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

    return x_train_a, x_test_a, x_train_v, x_test_v, y_train, y_test, split_stats

In [17]:
train_test_split_ratio = 0.7
X_train_aud, X_test_aud, X_train_vid, X_test_vid, Y_train, Y_test, stats = process_concat_data(concat_data, vid_features, aud_features, train_test_split_ratio)

In [18]:
for key in stats:
    print(key,':',stats[key])

Split ratio : 0.7140288273813417
IDs in Train : ['P318', 'P315', 'N254', 'P316', 'P246', 'P245', 'N260', 'N251', 'P244', 'N332', 'N334', 'P317', 'P314', 'P237', 'N261', 'N258', 'N253', 'N256', 'N255', 'P264', 'P240', 'N252', 'N249', 'P248']
IDs in Test : ['P265', 'N335', 'P203', 'P239', 'N333', 'N262', 'N257', 'N250', 'P263', 'P243', 'P242']
Number of Ns in train : 12
Number of Ps in train : 12
Number of Ns in test : 5
Number of Ps in test : 6


In [19]:
print('Train RB: ', concat_data.loc[ concat_data["pair_id"].isin([ d for d in stats['IDs in Train'] if d[0] == "P" ])].shape[0] / concat_data.loc[ concat_data['pair_id'].isin(stats['IDs in Train'])  ].shape[0] )
print('Test RB: ', concat_data.loc[ concat_data["pair_id"].isin([ d for d in stats['IDs in Test'] if d[0] == "P" ])].shape[0] / concat_data.loc[ concat_data['pair_id'].isin(stats['IDs in Test'])  ].shape[0] )

Train RB:  0.5490825838103025
Test RB:  0.5034951900305421


In [15]:
# device = torch.device("cuda")
# aud_model.to(device)
# vid_model.to(device)
aud_logits_train = aud_model(X_train_aud)
vid_logits_train = vid_model(X_train_vid)

aud_logits_test = aud_model(X_test_aud)
vid_logits_test = vid_model(X_test_vid)

In [16]:
vid_logits_train.shape

torch.Size([301139, 1])

In [17]:
Y_train.shape

torch.Size([301139, 1])

In [18]:
X_concat_train = np.concatenate((aud_logits_train.detach().numpy(), vid_logits_train.detach().numpy()), axis=1)
X_concat_test = np.concatenate((aud_logits_test.detach().numpy(), vid_logits_test.detach().numpy()), axis=1)

In [19]:
models = {
    "Logistic_Regression": {'function': LogisticRegression(), 'param_grid': [ {'Logistic_Regression__solver': ['saga'], 'Logistic_Regression__penalty': ['l1','l2'],
                                                                                'Logistic_Regression__tol': [1e-2, 1e-3, 1e-4, 1e-5]}]}, 
    
    # "SVM": {'function': SVC(), 'param_grid': [{'SVM__kernel': ['poly'], 'SVM__degree': [2,3,4,5,6]},
    #                                             {'SVM__kernel': ['linear', 'rbf', 'sigmoid']}]}, 
    
    "RF": {'function': RandomForestClassifier(), 'param_grid': [{'RF__n_estimators': [2,5,10,20],'RF__criterion': ['gini', 'entropy', 'log_loss'], 'RF__max_depth': [1,2,3,4,5,10,15],
                                                                                        'RF__max_features': [0.1, 0.2, 'sqrt', 'log2']}]},
    "KNN":{'function': KNeighborsClassifier(), 'param_grid': [{'KNN__n_neighbors': [1,2,3,4,5]}]}
}

In [20]:
#classifier = sklearn.svm.SVC(kernel = 'rbf',C = 10,gamma=0.1)
#classifier = lm.LogisticRegression()
#classifier = nei.KNeighborsClassifier(n_neighbors=7)
#classifier = en.RandomForestClassifier(n_estimators=15, max_depth=50)
for name, items in models.items():
    pipe = Pipeline([
        ('sc', preprocessing.StandardScaler()),
        (name, items['function'])
    ])
    best_model = GridSearchCV(estimator=pipe, param_grid = items['param_grid'])
    best_model.fit(X_concat_train, Y_train.ravel())
    Pred = best_model.predict(X_concat_test)
    print(name, accuracy_score(Y_test, Pred))

Logistic_Regression 0.5163577696454571
RF 0.513020894817613
KNN 0.4983512375555818


In [21]:
MLP_param_grid = {
    'MLP__hidden_layer_sizes': [(10,10), (5,10,5),(5,5,5), (5,10,5), (10,)],
    'MLP__activation': ['tanh', 'relu'],
    'MLP__solver': ['sgd', 'adam'],
    'MLP__alpha': [0.0001, 0.01],
    'MLP__learning_rate': ['adaptive'],
    'MLP__max_iter': [100]
}
mlp_pipe = Pipeline([
    ('sc', preprocessing.StandardScaler()),
    ('MLP', MLPClassifier())
])

In [24]:
MLP_best_classifier = GridSearchCV(estimator=mlp_pipe, param_grid = MLP_param_grid)
MLP_best_classifier.fit(X_concat_train, Y_train.ravel())

GridSearchCV(estimator=Pipeline(steps=[('sc', StandardScaler()),
                                       ('MLP', MLPClassifier())]),
             param_grid={'MLP__activation': ['tanh', 'relu'],
                         'MLP__alpha': [0.0001, 0.01],
                         'MLP__hidden_layer_sizes': [(10, 10), (5, 10, 5),
                                                     (5, 5, 5), (5, 10, 5),
                                                     (10,)],
                         'MLP__learning_rate': ['adaptive'],
                         'MLP__max_iter': [100],
                         'MLP__solver': ['sgd', 'adam']})

In [26]:
Pred = MLP_best_classifier.predict(X_concat_test)
print('MLP', accuracy_score(Y_test, Pred))

MLP 0.5190807854247826


In [27]:
print(classification_report(Y_test, Pred))


              precision    recall  f1-score   support

         0.0       0.53      0.16      0.25     66459
         1.0       0.51      0.86      0.64     68603

    accuracy                           0.52    135062
   macro avg       0.52      0.51      0.44    135062
weighted avg       0.52      0.52      0.45    135062



In [28]:
print(confusion_matrix(Y_test, Pred))

[[10711 55748]
 [ 9661 58942]]
